In [1]:
import pandas as pd
import numpy as np
import scipy.io
import math
from collections import Counter
import copy
import warnings
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from numpy import argmax
warnings.filterwarnings("ignore")

In [2]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
def rms(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    mean = (sum / (float)(n))
    root = math.sqrt(mean)
    return root
def mav(arr):
    n = len(arr)
    mav = sum((abs(np.array(arr))))/(float)(n)
    return mav
def var(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    result = (sum / (float)(n))
    return result
def wl(arr):
    n = len(arr)
    sum = 0
    for i in range(1, n):
        sum += abs(arr[i] - arr[i - 1])
    return sum
def iav(arr):
    return sum(abs(np.array(arr)))
def mean(arr):
    return np.sum(arr)/len(arr)
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):

            pretty(value, indent + 2)
            print('-----------------------')
        else:
            print('\t' * (indent + 2) + str(value))
            print(" ")
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
def extractSubject(name):
    ex1Path = 'DB1/' + name + '/' + name + '_A1_E1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    emg = ex1['emg']
    EMGdf = pd.DataFrame.from_dict(emg)
    stimulus = ex1['restimulus']

    ex2Path = 'DB1/' + name + '/' + name + '_A1_E2.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    emg2 = ex2['emg']
    EMGdf2 = pd.DataFrame.from_dict(emg2)
    stimulus2 = ex2['restimulus']

    ex3Path = 'DB1/' + name + '/' + name + '_A1_E3.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    emg3 = ex3['emg']
    EMGdf3 = pd.DataFrame.from_dict(emg3)
    stimulus3 = ex3['restimulus']

    Movements = {}
    for m in range(1, 51):
        if (m < 11):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf
        elif (m < 28):
            movementIndices = np.where(stimulus2 == (m - 10))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf2
        else:
            movementIndices = np.where(stimulus3 == (m - 27))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf3

        Electrodes = {}
        for e in range(1, 11):
            temp = {}
            for r in range(1, 7):
                startIndex = repetitions[r - 1][0]
                LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
                df = EMG.iloc[startIndex:LastIndex, e - 1]
                df.reset_index(drop=True, inplace=True)
                narray = df.to_numpy(dtype=None, copy=False)
                temp["R{0}".format(r)] = narray
            Electrodes["Electrode{0}".format(e)] = temp
        Movements["Movement{0}".format(m)] = Electrodes
    return Movements


In [3]:
final_df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                                 'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                                 'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                                 'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                                 'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                                 'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                                 'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                                 'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                                 'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                                 'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                                 'Train'})
for s in (1,2,3,4,5,7,9,11,12,13,16,17,18,19,21,23,24,25,26,27):
# for s in range(1,28):
    subject = 'S' + str(s)
    dff = pd.DataFrame.from_dict(extractSubject(subject))
    df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                           'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                           'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                           'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                           'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                           'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                           'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                           'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                           'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                           'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                           'Train'})
    for e in range(1, 11):
        electrode = 'Electrode' + str(e)
        i=0
        for m in range(1,51):
            M = dff['Movement'+str(m)][electrode]
            for r in range(1, 7):
                rep = "R" + str(r)
                if (r in [1, 3, 4, 6]):
                    train = 1
                else:
                    train = 0
                for x in range(0, len(M[rep]), 20):
                    df.at[i, 'RMS' + str(e)] = rms(M[rep][x:x + 50])
                    df.at[i, 'MAV' + str(e)] = mav(M[rep][x:x + 50])
                    df.at[i, 'VAR' + str(e)] = var(M[rep][x:x + 50])
                    df.at[i, 'WL' + str(e)] = wl(M[rep][x:x + 50])
                    df.at[i, 'IAV' + str(e)] = iav(M[rep][x:x + 50])
                    df.at[i, 'Train'] = train
                    df.at[i, 'Movement'] = m
                    i += 1
    final_df = final_df.append(df, ignore_index=True)

DB1/S1/S1_A1_E1.mat
DB1/S2/S2_A1_E1.mat
DB1/S3/S3_A1_E1.mat
DB1/S4/S4_A1_E1.mat
DB1/S5/S5_A1_E1.mat
DB1/S7/S7_A1_E1.mat
DB1/S9/S9_A1_E1.mat
DB1/S11/S11_A1_E1.mat
DB1/S12/S12_A1_E1.mat
DB1/S13/S13_A1_E1.mat
DB1/S16/S16_A1_E1.mat
DB1/S17/S17_A1_E1.mat
DB1/S18/S18_A1_E1.mat
DB1/S19/S19_A1_E1.mat
DB1/S21/S21_A1_E1.mat
DB1/S23/S23_A1_E1.mat
DB1/S24/S24_A1_E1.mat
DB1/S25/S25_A1_E1.mat
DB1/S26/S26_A1_E1.mat
DB1/S27/S27_A1_E1.mat


In [11]:
features = {'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
            'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
            'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
            'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
            'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
            'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
            'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
            'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
            'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
            'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10', }


print(final_df)

            VAR6    IAV6      MAV4      RMS7     IAV8      VAR3 Train     WL4  \
0       0.000006    0.12   0.00245  0.114243   2.6584  0.001387     1  0.0025   
1       0.000007  0.1299  0.014282  0.193561   3.2149  0.014163     1  0.0879   
2       0.000007  0.1299   0.02314  0.299421     4.16   0.02708     1  0.1736   
3        0.00001  0.1472  0.023586  0.361852   4.2627   0.02606     1  0.1977   
4        0.00001  0.1472   0.01117  0.323584   3.7084  0.010091     1  0.1295   
...          ...     ...       ...       ...      ...       ...   ...     ...   
114698  0.000579  1.0521  0.005218  0.253062  35.9841   0.00249     1  0.0695   
114699  0.000524   0.979    0.0036  0.277815  35.5469  0.000017     1   0.055   
114700  0.000133  0.4365   0.00345   0.18368  30.8547    0.0093     1    0.06   
114701  0.000036  0.1726  0.003746  0.095783  23.1103  0.018864     1  0.0425   
114702  0.000011  0.0581  0.003847  0.038778  10.9545  0.038364     1  0.0175   

            MAV5    WL10  .

In [52]:
X_train = final_df[final_df['Train'] == 1].loc[:, features]
scalar =  StandardScaler()
scalar = scalar.fit(X_train)
X_train = scalar.transform(X_train)
X_test = final_df[final_df['Train'] == 0].loc[:, features]
X_test = scalar.transform(X_test)

y_train = final_df[final_df['Train'] == 1].loc[:, {'Movement'}]
y_test = final_df[final_df['Train'] == 0].loc[:, {'Movement'}]
y_train = np.asarray(y_train).astype('float64')
y_test = np.asarray(y_test).astype('float64')



In [53]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
y_test = y_test.to_numpy()
y_test_new = [most_frequent(y_test[x:x + 26]) for x in range(0, len(y_test), 26)]
y_predicted_new = [most_frequent(y_pred[x:x + 26]) for x in range(0, len(y_pred), 26)]
accuracy_modified = accuracy_score(y_test_new, y_predicted_new)
print(accuracy_modified)


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

In [51]:
print(accuracy_modified)
print(y_pred)
print(y_test)
y_predicted_new = [most_frequent(y_pred[x:x + 26]) for x in range(0, len(y_pred), 26)]
# y_test = y_test.to_numpy()
y_test_new =  [most_frequent(y_test[x:x + 26]) for x in range(0, len(y_test), 26)]
accuracy_modified = accuracy_score(y_test_new, y_predicted_new)
print(accuracy_modified)
# # yhat = model.predict(X_test)
# print(predictions)
# predictions = argmax(predictions, axis=-1).astype('int')
# print(predictions.shape)
# acc = accuracy_score(y_test, predictions)
# print('Accuracy: %.3f' % acc)

0.0
[17.  1.  1. ... 50. 50. 24.]
[[ 1.]
 [ 1.]
 [ 1.]
 ...
 [50.]
 [50.]
 [50.]]


TypeError: unhashable type: 'numpy.ndarray'

In [ ]:
# print(y_train)
# input = Input(shape =(50,))
# L1 = Dense(500, activation='tanh')(input)
# L2 = Dense(50, activation='sigmoid')(L1)
# ouput = Dense(50, activation='softmax')(L2)
# model = Model(input, ouput)
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
# model.fit(X_train, y_train, epochs=1000)
# print(model.evaluate(X_test, y_test))
# predictions = model.predict(X_test)